# Etape 1 : Les Features

<font color='grey'>Imports dont on a besoin pour cet exercice :</font>

In [8]:
from bigml.api import BigML
from pandas import read_csv
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_curve, auc, roc_auc_score

<font color='grey'>Charger vos identifiants kaggle et bigml :</font>

In [9]:
%load_ext dotenv
%dotenv

<font color='grey'>Importer les fichiers *"fulltrain"* et *"test"* de Kaggle :</font>

In [10]:
fulltrain = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)
test = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)

<font color='grey'>Indiquer le "project key" présent de BigML (il faut avoir créer au préalable un projet)</font>

In [11]:
api= BigML (project='project/5d94a42b42129f2e6200010e')

<font color='grey'>Ajouter des features aux datasets "fulltrain" et "test" de Kaggle :</font>

In [5]:
def features_credit(df):
    df.fillna(0,inplace=True)
    df['IncomePerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
    df.loc[df.age > 80, 'isOld'] = '1' 
    df.loc[df.age <= 80, 'isOld'] = '0'
    df['MonthlyDebt'] = df['MonthlyIncome'] * df['DebtRatio']
    df['MonthlyBalance'] = df['MonthlyIncome'] - df['MonthlyDebt']
    df['DebtPerPerson'] = df['MonthlyDebt'] / ( df['NumberOfDependents'] + 1 )
    df['BalancePerPerson'] = df['MonthlyBalance'] / ( df['NumberOfDependents'] + 1 )
    df['NumberOfTime30-89DaysPastDueNotWorse'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTime60-89DaysPastDueNotWorse']
    df['NumbersOfOpen-NumberRealEstate'] = df['NumberOfOpenCreditLinesAndLoans'] - df['NumberRealEstateLoansOrLines']
    return df


In [6]:
data_sets = fulltrain, test

for df in data_sets :
    features_credit(df)

<font color='grey'>Faire un ***list(dataframe)*** pour s'assurer que les features ont bien été ajoutés :</font>

In [7]:
list(fulltrain)

['RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents',
 'SeriousDlqin2yrs',
 'IncomePerPerson',
 'isOld',
 'MonthlyDebt',
 'MonthlyBalance',
 'DebtPerPerson',
 'BalancePerPerson',
 'NumberOfTime30-89DaysPastDueNotWorse',
 'NumbersOfOpen-NumberRealEstate']

In [6]:
list(test)

['SeriousDlqin2yrs',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents',
 'IncomePerPerson',
 'isOld',
 'MonthlyDebt',
 'MonthlyBalance',
 'DebtPerPerson',
 'BalancePerPerson',
 'NumberOfTime30-89DaysPastDueNotWorse',
 'NumbersOfOpen-NumberRealEstate']

<font color='grey'>- Le trainfull modifié va nous servir à faire le split et sera notre base de travail (créer le modèle et l'entrainer).</font><br>
<font color='grey'> - Le test modifé est le fichier qu'on va modifier afin de le soumettre à Kaggle.</font><br><br>
<font color='grey'>Exporter les fichiers modifiés en csv :</font>


In [ ]:
export_fulltrain_csv = fulltrain.to_csv ('data/fulltrain_dataframe.csv', header=True)
export_test_csv = test.to_csv ('data/test_dataframe.csv', header=True)

# Etape 2: Le Split

<font color='grey'>Effectuer le split sur le fichier trainfull modifié qu'on vient d'exporter en csv *(fulltrain_dataframe.csv)* :</font>

<font color='grey'>Créer la source *(ici fulltrain_dataframe.csv)*:</font>

In [ ]:
source = api.create_source('fulltrain_dataframe.csv')
api.ok(source)

<font color='grey'>Créer le dataset à partir de la source :</font>

In [ ]:
fulltrain_dataset = api.create_dataset(source)
api.ok(fulltrain_dataset)

<font color='grey'>Réaliser le split 80/20 :<br>
**attention de bien indiquer le seed sinon le split se fera de façon aléatoire à chaque fois** </font>

In [ ]:
train80 = api.create_dataset(
    fulltrain_dataset, {"name": "train80 | TrainFull | Training",
                     "sample_rate": 0.8, "seed": "my seed"})
valid20 = api.create_dataset(
    fulltrain_dataset, {"name": "valid20 | TrainFull | Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [ ]:
api.ok(train80)

In [ ]:
api.ok(valid20)

<font color='grey'>On peut exporter les datasets train80 et valid20 en csv :<br>

In [ ]:
api.download_dataset(train80,
    filename='data/train80.csv')

In [ ]:
api.download_dataset(valid20,
    filename='data/valid20.csv')

# Etape 3 : Le Modèle

<font color='grey'>Créer le modèle avec le dataset train80 en indiquant la *"target variable"*(= ce qu'on veut prédire) :<br>
Ici le modèle choisi est **"ensemble"** :</font>

In [ ]:
ensemble = api.create_ensemble(train80, {"objective_field":"SeriousDlqin2yrs"})
api.ok(ensemble)

<font color='grey'>Créer le batch prediction sur le val20 dataset:</font>

In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, valid20,{"all_fields": True, "confidence": True, "probabilities": True})
api.ok(batch_prediction)

Il nous faut faire le batch prediction sur le fichier test. Pour cela :

<font color='grey'>Créer une source pour le test set modifié :</font>

In [ ]:
test_source = api.create_source('data/test_dataframe.csv')

In [ ]:
api.ok(test_source)

<font color='grey'>Créer le dataset à partir de cette source :</font>

In [ ]:
test_dataset = api.create_dataset(test_source)

In [ ]:
api.ok(test_dataset)

<font color='grey'>Evaluer le modèle créé sur le trainfull_dataset :</font>

In [ ]:
evaluation = api.create_evaluation(ensemble, trainfull_dataset)
api.ok(evaluation)

<font color='grey'>Imprimer le résulat de l'évaluation :</font>

In [ ]:
print("accuracy:")
api.pprint(evaluation['object']['result']['model']['accuracy'])
print("auc:")
api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])
print("f_measure:")
api.pprint(evaluation['object']['result']['model']['per_class_statistics'][0]['f_measure'])

<font color='grey'>Faire le batch prediction sur le dataset test :</font>

In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, test_dataset,{"all_fields": True, "confidence": True, "probabilities": True})
api.ok(batch_prediction)

<font color='grey'>Télécharger le batch prediction en fichier .csv afin de pouvoir le préparer pour le soumettre à Kaggle :</font>

In [ ]:
api.download_batch_prediction(batch_prediction,
    filename='data/prediction_credit_final.csv')